#### The Bridge 2
https://www.codingame.com/ide/puzzle/the-bridge-episode-2

In [2]:
from overwrite_input import *
init_data('the_bridge_5.txt');

In [11]:
from sre_parse import State
import sys
import math
import queue
from typing import List, Set, Tuple, Union, Iterator
from collections import deque

def log(*args):
    print(*args, file=sys.stderr, flush=True)

m = int(input())  # the amount of motorbikes to control
min_surviving_bikes = int(input())  # the minimum amount of motorbikes that must survive
lines = [input()+'.' for _ in range(4)] # add dot at the end to match speed hack in dead checks
bridge_length = len(lines[0])-1

initial_bike_count = 0
command_strings = 'SPEED, UP, DOWN, WAIT, JUMP, SLOW'.split(', ')
commands = [str(c) for c in range(len(command_strings))]
SPEED, UP, DOWN, WAIT, JUMP, SLOW = commands

Bikes = Tuple[bool, bool, bool, bool]
StateCommand = Tuple[int, Bikes, int, str] # x, bikes, speed, command
LeastLossesChain = Tuple[int, str] # survivor_count, commands

################################################################################################################

def bike_after_cmd(state_cmd: StateCommand, bike_y: int) -> bool:
    x, bikes, speed, cmd = state_cmd
    if not bikes[bike_y]: return False # dead bikes
    if x + 1 >= bridge_length: return True # check finish line

    new_y = bike_y
    if cmd == UP: new_y -= 1
    if cmd == DOWN: new_y += 1
    if cmd == SPEED and speed < 49: speed += 1
    if cmd == SLOW and speed > 0: speed -= 1

    # limit speed to avoid lines out of bounds
    if x + speed >= bridge_length: 
        speed = bridge_length - x # without -1 because lines end with an extra .

    # jumping
    if cmd == JUMP:
        return lines[bike_y][x + speed] == '.'

    # check holes on new lane
    for i in range(1 if speed else 0, speed + 1):
        if lines[new_y][x + i] == '0': return False

    # check holes on OLD lane
    if new_y != bike_y:
        for i in range(1, speed):
            if lines[bike_y][x + i] == '0': return False

    return True

def state_after_command(state_cmd: StateCommand) -> StateCommand:
    x, bikes, speed, cmd = state_cmd
    if cmd == '': return state_cmd

    bikes: Bikes = tuple(
        bike_after_cmd(state_cmd, bike_y) 
        for bike_y, _ in enumerate(bikes)
    )
    
    # shift bikes if up/down
    if state_cmd[3] == UP: bikes = (*bikes[1:], False)
    if state_cmd[3] == DOWN: bikes = (False, *bikes[:3])

    # adjust speed
    if cmd == SPEED and speed < 49: speed += 1
    if cmd == SLOW and speed > 0: speed -= 1

    # adjust x
    x += speed

    return x, bikes, speed, cmd

def get_next_commands(state_cmd: StateCommand) -> Iterator[StateCommand]:
    x, bikes, speed, current_cmd = state_cmd
    for cmd in commands:
        if cmd == UP and bikes[0]: continue
        if cmd == DOWN and bikes[3]: continue
        if speed == 0:
            if cmd in (JUMP, WAIT, SLOW): continue
            if cmd == UP and current_cmd == DOWN: continue
            if cmd == DOWN and current_cmd == UP: continue
        yield x, bikes, speed, cmd

def get_command_chain(init_bikes: Bikes, init_speed: int):
    # params for commands: x, bikes, speed, cmd
    root_cmd = (0, init_bikes, init_speed, '')
    return dfs_winning_command(set(), root_cmd, '', (0, ''))

def dfs_winning_command(
    visited: Set[StateCommand], 
    state_cmd: StateCommand, 
    cmd_chain: str, 
    best_cmd_chain: LeastLossesChain
) -> LeastLossesChain:
    visited.add(state_cmd)

    state_after = state_after_command(state_cmd)
    cmd_chain += state_cmd[3]
    surviving_bike_count = sum(state_after[1])
    if surviving_bike_count < min_surviving_bikes:
        return

    # Store command if better than last one
    if state_after[0] >= bridge_length:
        return surviving_bike_count, cmd_chain
    
    for next_cmd in get_next_commands(state_after):
        if next_cmd not in visited:
            chain = dfs_winning_command(visited, next_cmd, cmd_chain, best_cmd_chain)
            if chain is None: continue

            # return early if no-sacrifice trail
            if chain[0] == initial_bike_count:
                return chain

            # store best chain
            if chain[0] > best_cmd_chain[0]:
                best_cmd_chain = chain

    if best_cmd_chain[0] < min_surviving_bikes: return

    return best_cmd_chain

################################################################################################
# game loop
turn_id = 0
command_chain = ''
while True:
    speed = int(input())
    bikes = [False for _ in range(4)]
    for i in range(m):
        bx, by, bike_alive = [int(j) for j in input().split()]
        if bike_alive: bikes[by] = True

    if turn_id == 0:
        initial_bike_count = sum([b for b in bikes])
        _, command_chain = get_command_chain(tuple(bikes), speed)

    # A single line containing one of 6 keywords: SPEED, SLOW, JUMP, WAIT, UP, DOWN.
    cmd_id = int(command_chain[turn_id]) if turn_id < len(command_chain) else command_strings.index('WAIT')
    print(command_strings[cmd_id])
    turn_id += 1


WAIT


Exception: End of input